In [2]:
import openmeteo_requests

import requests_cache
import pandas as pd

from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from retry_requests import retry
import pickleshare

In [30]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": 52.52,
    "longitude": 13.41,
    "start_date": "1999-12-01",
    "end_date": "1999-12-31",
    "hourly": "temperature_2m",
    "daily": "temperature_2m_mean"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
    start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
    end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
    freq=pd.Timedelta(seconds=hourly.Interval()),
    inclusive="left",
    normalize=True  # Setze normalize=True, um nur das Datum zu erhalten
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data=hourly_data)

# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_temperature_2m_mean = daily.Variables(0).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
    start=pd.to_datetime(daily.Time(), unit="s", utc=False),
    end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=False),
    freq=pd.Timedelta(seconds=daily.Interval()),
    inclusive="left",
    normalize=True  # Setze normalize=True, um nur das Datum zu erhalten
)}
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean

daily_dataframe = pd.DataFrame(data=daily_data)

# Set the index of the DataFrame to the date only
daily_dataframe.set_index("date", inplace=True)

return daily_dataframe

            temperature_2m_mean
date                           
1999-12-01             7.916082
1999-12-02             4.857750
1999-12-03             5.553583
1999-12-04             3.514000
1999-12-05             1.926500
1999-12-06             1.761917
1999-12-07             7.126501
1999-12-08             5.530667
1999-12-09             7.720250
1999-12-10             6.428583
1999-12-11             5.330667
1999-12-12             8.880667
1999-12-13             3.993166
1999-12-14             2.518167
1999-12-15             0.609833
1999-12-16             0.026500
1999-12-17             1.303583
1999-12-18             1.307750
1999-12-19            -0.765167
1999-12-20             0.378583
1999-12-21            -1.229750
1999-12-22            -1.813083
1999-12-23            -1.073500
1999-12-24             1.334833
1999-12-25             4.370250
1999-12-26             1.639000
1999-12-27             1.791083
1999-12-28             0.593167
1999-12-29             0.559833
1999-12-